<a href="https://colab.research.google.com/github/glee1228/net_VLad_keras/blob/master/netVLAD_triplet_keras_ldh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 코랩 연동

In [1]:
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from keras.applications.nasnet import NASNetMobile
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import VGG16
import tensorflow as tf
import keras
from keras import layers

Using TensorFlow backend.


In [2]:
!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab
tb = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://a0b3bc0f.ngrok.io


In [3]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
    drive.mount('/content/gdrive')
    print('Google Drive is mounted\n')
else:
    print('Google Drive is already mounted\n')

Google Drive is already mounted



In [0]:
tf.set_random_seed(777)

## 데이터 전처리 후 저장

In [0]:
prof_team = False

In [0]:
if prof_team :
  ckpt_path = '/content/gdrive/My Drive/AILeader_Dataset/Checkpoint_jw'
  train_path = './gdrive/My Drive/AILeader_Dataset/train'
  test_path = './gdrive/My Drive/AILeader_Dataset/test'
  
else :
  ckpt_path = './gdrive/My Drive/ckpt'
  train_path = './gdrive/My Drive/train'
  test_path = './gdrive/My Drive/test'

In [0]:
os.environ['CUDA_VISIBLE_DEVICES']= '0'
data_dir = './gdrive/My Drive/AILeader_Dataset'
resized_data_dir = './gdrive/My Drive/resized_Dataset'
resized_data_path = ['./gdrive/My Drive/resized_Dataset/train_x.npy','./gdrive/My Drive/resized_Dataset/train_y.npy']

In [8]:
! ls -a ./gdrive/My\ Drive/train

 AI	        dorm		   'Front door Sejong'	'Stone statue'
'Clock tower'  'Front door Child'   Museum


## NetVLAD

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
import keras
import code
import numpy as np
class NetVLADLayer(Layer):

    def __init__( self, num_clusters, **kwargs ):
        self.num_clusters = num_clusters
        super(NetVLADLayer, self).__init__(**kwargs)

    def build( self, input_shape ):
        self.K = self.num_clusters
        self.D = input_shape[-1]


        self.kernel = self.add_weight( name='kernel',
                                    shape=(1,1,self.D,self.K),
                                    initializer='uniform',
                                    trainable=True )

        self.bias = self.add_weight( name='bias',
                                    shape=(1,1,self.K),
                                    initializer='uniform',
                                    trainable=True )

        self.C = self.add_weight( name='cluster_centers',
                                shape=[1,1,1,self.D,self.K],
                                initializer='uniform',
                                trainable=True)


    def call( self, x ):
        s = K.conv2d( x, self.kernel, padding='same' ) + self.bias
        a = K.softmax( s )
        self.amap = K.argmax( a, -1 )
 
        a = K.expand_dims( a, -2 ) 
        v = K.expand_dims(x, -1) + self.C 
        v = a * v
        v = K.sum(v, axis=[1, 2])
        v = K.permute_dimensions(v, pattern=[0, 2, 1])

        v = K.l2_normalize( v, axis=-1 )
        v = K.batch_flatten( v )
        v = K.l2_normalize( v, axis=-1 )

        # return [v, self.amap]
        return v

    def compute_output_shape( self, input_shape ):
        return (input_shape[0], self.K*self.D )

    def get_config( self ):
        pass
        # base_config = super(NetVLADLayer, self).get_config()
        # return dict(list(base_config.items()))

        # As suggested by: https://github.com/keras-team/keras/issues/4871#issuecomment-269731817
        config = {'num_clusters': self.num_clusters}
        base_config = super(NetVLADLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [0]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    
    print('y_pred.shape = ',y_pred)
    
    total_lenght = y_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

## base model 

In [0]:
batch_size= 32
input_shape = (128,128,3)

In [0]:
from keras import Model, Input
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.optimizers import SGD,Adam
from keras.models import Sequential

def initialize_model(backbone= None, input_shape=input_shape, use_pretrained = 'imagenet'):
  base_model = None
  dim = 0
  base_model = backbone(input_shape=input_shape, weights=use_pretrained, include_top= False)
  x = base_model.output
  dim = x[-1].get_shape().as_list()[2]
  
  return base_model, dim



In [0]:
## VGG16 or Xception or DenseNet121 or 
## DenseNet169 or DenseNet201 or NASNetMobile or 
## ResNet50 or InceptionResNetV2
model_name = VGG16  

In [14]:
base_model, dim =initialize_model(model_name, input_shape=input_shape)

print('dim : ',dim)

Instructions for updating:
Colocations handled automatically by placer.
dim :  512


## Triplet Sampler

In [37]:
dir_pathes = []
i =0
for root,dir,file in os.walk(train_path):
  if i>=1:
    dir_pathes.append(root)
  i+=1
print(dir_pathes) # class dir path

['./gdrive/My Drive/train/Front door Child', './gdrive/My Drive/train/Museum', './gdrive/My Drive/train/AI', './gdrive/My Drive/train/Front door Sejong', './gdrive/My Drive/train/Stone statue', './gdrive/My Drive/train/Clock tower', './gdrive/My Drive/train/dorm']


In [24]:
import numpy as np
import random
from PIL import Image

triplet_num = 50
img_width = 128
img_height = 128
total_len =len(dir_pathes)*triplet_num
print('total triplets length : {}'.format(total_len))
triplets = np.zeros((total_len,3,img_width,img_height,3))

triplets_len = 0
for i in range(len(dir_pathes)):
    # set the files of anchor and positive images
    anc_pos_files = [f for f in sorted(os.listdir(dir_pathes[i]))]
    for iteration in range(triplet_num):

        # set the files of negative images
        j = random.choice([num for num in range(len(dir_pathes)) if num not in [i]])
        nega_files = [o for o in sorted(os.listdir(dir_pathes[j]))]
        
        # get anchor and positive images as numpy array
        pair = np.random.randint(0,len(anc_pos_files),2)
        anchor = np.array(Image.open('{0}/{1}'.format(dir_pathes[i],anc_pos_files[pair[0]])).resize((img_width,img_height)))
          
        positive = np.array(Image.open('{0}/{1}'.format(dir_pathes[i],anc_pos_files[pair[1]])).resize((img_width,img_height)))

        # get negative images as numpy array
        nega_idx = np.random.randint(len(nega_files))
        negative = np.array(Image.open('{0}/{1}'.format(dir_pathes[j],nega_files[nega_idx])).resize((img_width,img_height)))
        
        # if image channel !=3 , resample triplets
        if anchor.shape[2]!=3 or positive.shape[2]!=3 or negative.shape[2]!=3:
          # get anchor and positive images as numpy array
          pair = np.random.randint(0,len(anc_pos_files),2)
          anchor = np.array(Image.open('{0}/{1}'.format(dir_pathes[i],anc_pos_files[pair[0]])).resize((img_width,img_height)))
          
          positive = np.array(Image.open('{0}/{1}'.format(dir_pathes[i],anc_pos_files[pair[1]])).resize((img_width,img_height)))

          # get negative images as numpy array
          nega_idx = np.random.randint(len(nega_files))
          negative = np.array(Image.open('{0}/{1}'.format(dir_pathes[j],nega_files[nega_idx])).resize((img_width,img_height)))
        # (optional)visualization
        # plot_triplet(triplet)
        
        triplets[triplets_len:triplets_len+1,0:1,:,:,:]=anchor/255.
        triplets[triplets_len:triplets_len+1,1:2,:,:,:]=positive/255.
        triplets[triplets_len:triplets_len+1,2:3,:,:,:]=negative/255.
        
        triplets_len+=1
        if triplets_len%20==0:
          print('{}/{}'.format(triplets_len,total_len))


total triplets length : 350
20/350
40/350
60/350
80/350
100/350
120/350
140/350
160/350
180/350
200/350
220/350
240/350
260/350
280/350
300/350
320/350
340/350


In [25]:
triplets.shape

(350, 3, 128, 128, 3)

In [0]:
triplets_dir = './gdrive/My Drive/triplets'
import os
if os.path.exists(triplets_dir)==False:
  os.mkdir(triplets_dir)
else:
  pass

In [29]:
# save triplets as numpy array
triplets_path = os.path.join(triplets_dir,'triplets.npy')
print('save file')
np.save(triplets_path,triplets)

save file


In [0]:
# triplets = np.load(triplets_path)

## DataLoader

In [33]:
# def extract_features(directory, sample_count):
#     features = np.zeros(shape=(sample_count, 4, 4, 512))
#     labels = np.zeros(shape=(sample_count))
#     generator = datagen.flow_from_directory(
#         directory,
#         target_size=(128, 128),
#         batch_size=batch_size,
#         class_mode='binary')
#     i = 0
#     for inputs_batch, labels_batch in generator:
#         features_batch = base_model.predict(inputs_batch)
        
#         features[i * batch_size : (i + 1) * batch_size] = features_batch
#         labels[i * batch_size : (i + 1) * batch_size] = labels_batch
#         i += 1
#         if i * batch_size >= sample_count:
#             break
#     return features, labels

# train_features, train_labels = extract_features(train_path, 500)

Found 749 images belonging to 7 classes.


ValueError: ignored

In [0]:
# import os
# if os.path.exists(resized_data_dir)==False:
#   os.mkdir(resized_data_dir)
# else:
#   pass

In [17]:
# np.save(resized_data_path[0],train_features)
# np.save(resized_data_path[1],train_labels)

NameError: ignored

In [0]:
# train_features = np.load(resized_data_path[0])
# train_labels = np.load(resized_data_path[1])

In [20]:
# print(train_features.shape)
# print(train_labels.shape)

(500, 4, 4, 512)
(500,)


## Loss and Entire Model

In [0]:
##https://codepad.co/snippet/triplet-loss-in-keras-tensorflow-backend
from keras.models import Model
from keras.layers import Input, concatenate,merge
from keras.optimizers import Adam

ALPHA = 0.2  # Triplet Loss Parameter

def triplet_loss(x):
    anchor, positive, negative = x

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), 1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), 1)

    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), ALPHA)
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0), 0)

    return loss

def load_model(input_shape,base_model,net_vlad):
    anchor = Input(shape=input_shape)
    pos = Input(shape=input_shape)
    neg = Input(shape=input_shape)
    
    anchor_feature=base_model(anchor)
    pos_feature=base_model(pos)
    neg_feature=base_model(neg)
    
    anchor_vlad = net_vlad(anchor_feature)
    pos_vlad = net_vlad(pos_feature)
    neg_vlad = net_vlad(neg_feature)
    
    loss = merge([anchor_vlad, pos_vlad, neg_vlad],mode=triplet_loss, output_shape=(1,))
    
    model = Model(inputs=[anchor_example, positive_example, negative_example],
                  outputs=loss)
    
    model.compile(loss='mean_absolute_error', optimizer=Adam())
    return model
    

In [0]:
# from keras import metrics
# from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(rescale=1./255)
# batch_size = 20
# train_generator = datagen.flow_from_directory(train_path,
#                                        target_size=(128,128),
#                                        batch_size=batch_size,
#                                        class_mode='binary')

# inputs = None
# labels = None
# i = 0
# for input_batches, input_labels in train_generator:
#   if i==1:
#     break
#   else:
#     inputs=input_batches
#     labels=input_labels
#   i+=1

In [47]:
net_vlad = NetVLADLayer(num_clusters=9)
model = load_model((128,128,3),base_model,net_vlad)
output=model(tf.convert_to_tensor(inputs, np.float32))


TypeError: ignored

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
net_vlad_layer_8 (NetVLADLay (None, 4608)              9225      
Total params: 14,723,913
Trainable params: 14,723,913
Non-trainable params: 0
_________________________________________________________________
